<a href="https://colab.research.google.com/github/ramsrigouthamg/Supertranslate.ai/blob/main/Burn_Subtitles_Into_Video/Burn_Subtitles_to_video_using_Moviepy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## How to permanently add or burn subtitles to a video using Moviepy

For files (mp4 and srt) used in this repository check: https://github.com/ramsrigouthamg/Supertranslate.ai

Author: [Ramsri Goutham](https://twitter.com/ramsri_goutham) 

In [ ]:
!pip install moviepy==2.0.0.dev2
!pip install imageio==2.25.1

In [ ]:
!apt install imagemagick

In [ ]:
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

In [ ]:
!pip install pysrt==1.1.2

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

In [ ]:
# Place files in this path or modify the paths to point to where the files are
srtfilename = "/content/gdrive/My Drive/Burn_Subtitles_Into_Video/AI_SaaS.srt"
mp4filename = "/content/gdrive/My Drive/Burn_Subtitles_Into_Video/AI_SaaS.mp4"

In [ ]:
import sys
import pysrt
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip


def time_to_seconds(time_obj):
    return time_obj.hours * 3600 + time_obj.minutes * 60 + time_obj.seconds + time_obj.milliseconds / 1000


def create_subtitle_clips(subtitles, videosize,fontsize=24, font='Arial', color='yellow', debug = False):
    subtitle_clips = []

    for subtitle in subtitles:
        start_time = time_to_seconds(subtitle.start)
        end_time = time_to_seconds(subtitle.end)
        duration = end_time - start_time

        video_width, video_height = videosize
        
        text_clip = TextClip(subtitle.text, fontsize=fontsize, font=font, color=color, bg_color = 'black',size=(video_width*3/4, None), method='caption').set_start(start_time).set_duration(duration)
        subtitle_x_position = 'center'
        subtitle_y_position = video_height* 4 / 5 

        text_position = (subtitle_x_position, subtitle_y_position)                    
        subtitle_clips.append(text_clip.set_position(text_position))

    return subtitle_clips




In [ ]:

# Load video and SRT file
video = VideoFileClip(mp4filename)
subtitles = pysrt.open(srtfilename)

begin,end= mp4filename.split(".mp4")
output_video_file = begin+'_subtitled'+".mp4"

print ("Output file name: ",output_video_file)

# Create subtitle clips
subtitle_clips = create_subtitle_clips(subtitles,video.size)

# Add subtitles to the video
final_video = CompositeVideoClip([video] + subtitle_clips)

# Write output video file
final_video.write_videofile(output_video_file)


In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open(output_video_file,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)